In [93]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np                   # advanced math library
import matplotlib.pyplot as plt 
import seaborn as sns     # MATLAB like plotting routines
import random                        # for generating random numbers
#
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
#
import os
#
import pandas as pd
import tensorflow as tf
#
from tensorflow.keras.models import Model
#
from tensorflow.keras.layers import Dense, Flatten, Input, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
#
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from PIL import Image as im

#
print(tf.__version__)

from keras.datasets import mnist     # MNIST dataset is included in Keras
from keras.models import Sequential  # Model type to be used


from keras.layers.core import Dense, Dropout, Activation # Types of layers to be used in our model
from keras.utils import np_utils      

2.10.0


In [2]:
train_dir = "food_images/validation"
test_dir  = "food_images/evaluation"

target_size = (224,224)
batch_size = 1

In [3]:
# Data Generator
train_datagen = ImageDataGenerator(
#    rotation_range=40,
#    width_shift_range=0.2,
#    height_shift_range=0.2,
#    shear_range=0.2,
#    zoom_range=0.2,
#    horizontal_flip=True,
#    vertical_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='sparse')

Found 3430 images belonging to 11 classes.


In [4]:
test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,    
    class_mode='sparse')

Found 3347 images belonging to 11 classes.


In [5]:
images = np.zeros((len(train_generator),target_size[0],target_size[1],3))
for i in np.arange(0,len(train_generator)):
    images[i,:] = train_generator[i][0]

In [6]:
images = images.astype(np.uint8)

In [7]:
images[0]

array([[[151, 145, 145],
        [ 35,  25,  23],
        [ 39,  21,  17],
        ...,
        [130, 122, 111],
        [149, 143, 131],
        [141, 135, 123]],

       [[166, 162, 163],
        [ 71,  63,  61],
        [ 33,  19,  16],
        ...,
        [133, 125, 114],
        [145, 137, 126],
        [140, 132, 121]],

       [[183, 183, 185],
        [166, 162, 163],
        [ 29,  19,  18],
        ...,
        [144, 134, 124],
        [135, 125, 115],
        [143, 130, 121]],

       ...,

       [[ 15,  14,  19],
        [ 13,  12,  17],
        [  9,   8,  13],
        ...,
        [160, 104,  53],
        [156, 103,  49],
        [159, 106,  52]],

       [[ 16,  15,  20],
        [ 15,  14,  19],
        [  9,   8,  13],
        ...,
        [159, 103,  54],
        [162, 106,  55],
        [161, 106,  52]],

       [[ 15,  14,  19],
        [ 15,  14,  19],
        [  8,   7,  12],
        ...,
        [159, 103,  56],
        [166, 110,  61],
        [164, 107,  54]]

In [118]:
def ImageCompressor(array, tolerance):
    a = np.zeros(np.shape(array))
    for i in np.arange(0,3):
        u, s,v = np.linalg.svd(array[:,:,i], full_matrices=False)
        s = np.diag(s)
        to_cut = int(tolerance*np.shape(s)[0])
        #print(np.shape(u),np.shape(s),np.shape(v))
        u = u[:,:-to_cut]
        s = s[:-to_cut,:-to_cut]
        v = v[:-to_cut,:]
        #print(np.shape(u),np.shape(s),np.shape(v))
        a[:,:,i] = u @ s @ v
    a = np.clip(a,0,255)
    a = a.astype(np.uint8)
    new_img = im.fromarray(a)
    new_img.save('compressed_img.jpg')
    return a

In [114]:
img = im.open("full_hd_image.jpg")
hd_image = np.asarray(img)
#print(os.stat("full_hd_image.jpg").st_size)

479404


In [117]:
result = ImageCompressor(hd_image,0.80)